In [1]:
import numpy as np
from scipy import sparse
import os
from utils import optm
from utils.functions import *
import numpy as np
from scipy import sparse
from importlib import reload



I = np.identity(2)
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


Sz = sparse.csr_matrix(Sz)
Sx = sparse.csr_matrix(Sx)
Sy = sparse.csr_matrix(Sy)
I = sparse.csr_matrix(I)

h1 = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real


h2 = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real


h3 = -(sparse.kron(sparse.kron(Sz,Sz,format='csr'), sparse.kron(I,I,format='csr'),format='csr') 
       + sparse.kron(sparse.kron(Sx,Sx,format='csr'), sparse.kron(I,I,format='csr'),format='csr')
       + sparse.kron(sparse.kron(Sy,Sy,format='csr'), sparse.kron(I,I,format='csr'),format='csr') 
     ).real

h4 = -(sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sz,Sz,format='csr'),format='csr') 
       + sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sx,Sx,format='csr'),format='csr')
       + sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sy,Sy,format='csr'),format='csr') 
     ).real




h1_ = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real

h2_ = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real



h = h1 + h2

h_ = h1_ + h2_

on_site = h3/4 + h4/4

In [2]:
J1 = 0.3
X1 = torch.tensor((J1 * h + on_site).toarray())
X1_prime = X1 - torch.eye(X1.shape[0])*(torch.diag(X1).min())

X2 = torch.tensor((J1 * h_ + on_site).toarray())
X2_prime = X2 - torch.eye(X2.shape[0])*(torch.diag(X2).min())

In [6]:
model._params = torch.nn.Parameter(model._params.data)

NameError: name 'model' is not defined

In [7]:
reload(optm)
model, gl = optm.optim_matrix_symm([X1_prime, X2_prime], 10000, optm_method = torch.optim.SGD, seed = 10, lr = 0.01, init_params = ret.x)

NameError: name 'ret' is not defined

In [35]:
reload(optm)
def optimize_func(X, seed = 0):
    X = torch.stack(X)
    if X.ndim!=3:
        X = X[None,:]
    L, lps, E = optm.get_mat_status(X)
    model = optm.unitary_solver([lps,lps],syms=True, seed = seed)
    N_param = (model._params).shape[0]
    def func(params):
        assert len(params) == N_param, "inconsistent"
        model._params = torch.nn.Parameter(torch.tensor(params, dtype=torch.float64))
        y = model(X)
        return optm.loss_eig(y)[0].item()
    return func

In [36]:
func = optimize_func([X1_prime, X2_prime])

In [6]:
ret.x

NameError: name 'ret' is not defined

In [39]:
%%time
minimizer_kwargs = {"method": "BFGS"}
import scipy.optimize as optimize
ret = optimize.dual_annealing(func, bounds = bounds, restart_temp_ratio = 1e-5, maxiter = 2000)
print(ret)

     fun: 1.3000000715255737
 message: ['Maximum number of iteration reached']
    nfev: 24401
    nhev: 0
     nit: 2000
    njev: 57
  status: 0
 success: True
       x: array([0.99376934, 1.78421771, 3.25076314, 3.37049956, 5.34690431,
       1.82501702])
CPU times: user 1min 44s, sys: 1.9 s, total: 1min 46s
Wall time: 8.84 s


[6]


[6]


In [91]:
%%time
reload(optm)
# X = np.stack([X1_prime, X2_prime])
X = 2*np.random.rand(16*16).reshape(16,16)-1
X = X + X.T + 2 * np.eye(16)
func = optm.unitary_optm(X)
import scipy.optimize as optimize
bounds = [[0, 2*np.pi] for _ in range(6)]
ret = optimize.dual_annealing(func, bounds = bounds, restart_temp_ratio = 1e-5, visit = 2.65, initial_temp = 2*10**4, maxiter = 10000, callback = callbackF)
print(ret)

[6]
target value : 10.8269000718 in context : 1
target value : 10.8253462301 in context : 0
target value : 10.8230516672 in context : 1
target value : 10.8229134129 in context : 0
target value : 10.8037571976 in context : 1
target value : 10.8036413619 in context : 0
target value : 10.8034279203 in context : 1
target value : 10.7870932133 in context : 0
target value : 10.5119178074 in context : 1
target value : 10.5119178047 in context : 2
target value : 10.5119177412 in context : 2
target value : 10.5119177336 in context : 2
target value : 10.5119177336 in context : 2
target value : 10.5119177313 in context : 2
target value : 10.5119177308 in context : 2
target value : 10.5119177295 in context : 2
target value : 10.5119177284 in context : 2
target value : 10.5119177281 in context : 2
target value : 10.5119177281 in context : 2
target value : 10.4590907380 in context : 2
target value : 10.4590907368 in context : 2
target value : 10.4590907364 in context : 2
target value : 10.4590690485

In [87]:
reload(optm)
model, gl = optm.optim_matrix_symm([torch.tensor(X)], 10000, optm_method = torch.optim.SGD, seed = 10, lr = 0.01, init_params = ret.x)

target loss : 7.765

---------- iteration start ----------
iteration :    0   loss : 11.155
iteration : 1000   loss : 11.156
iteration : 2000   loss : 11.157
iteration : 3000   loss : 11.157
iteration : 4000   loss : 11.159
iteration : 5000   loss : 11.161
iteration : 6000   loss : 11.160
iteration : 7000   loss : 11.157
iteration : 8000   loss : 11.162
iteration : 9000   loss : 11.158

 -------------- results --------------
target loss      : 7.7654734812
loss before optm : 12.6343341692
loss after optm  : 11.1578897134


In [77]:
torch.linalg.eigvalsh(model(torch.tensor(X)))

tensor([[-1.5575, -1.3128, -0.9974, -0.9227, -0.4537, -0.3640, -0.1814, -0.0176,
          0.4268,  0.5763,  0.8236,  1.0566,  1.4807,  1.6735,  2.0359,  7.4246]],
       dtype=torch.float64, grad_fn=<LinalgEighBackward0>)

In [83]:
torch.linalg.eigvalsh(model.matrix @ torch.tensor(X) @ model.matrix.T)

tensor([-1.5575, -1.3128, -0.9974, -0.9227, -0.4537, -0.3640, -0.1814, -0.0176,
         0.4268,  0.5763,  0.8236,  1.0566,  1.4807,  1.6735,  2.0359,  7.4246],
       dtype=torch.float64, grad_fn=<LinalgEighBackward0>)

In [11]:
np.sum(np.linalg.eigvalsh(X)[:,-1])

1.3000000119209285

In [19]:
X.transpose((0,2,1))

array([[[ 5.96046446e-09,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  1.25000006e-01, -1.25000000e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00, -1.25000000e-01,  4.25000006e-01,
          0.00000000e+00,  1.50000000e-01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  1.50000000e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.0000000

In [14]:
X = np.array(X) 
X.shape


(2, 16, 16)